In [ ]:
import re
import pandas as pd

In [ ]:
f = open('WhatsAppChat.txt','r',encoding='utf-8')
# instead of WhatsAppChat.txt give the name of the txt file of the exported chat
data = f.read()
print(data)

In [ ]:
print(type(data))

In [ ]:
# giving the pattern of date to take it
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
message = re.split(pattern, data)[1:]
message

In [ ]:
dates = re.findall(pattern, data)
dates

In [ ]:
df = pd.DataFrame({'user_message': message, 'message_date': dates})

df['message_date'] = pd.to_datetime(
    df['message_date'], format='%d/%m/%y, %H:%M - ')
df.rename(columns={'message_date': 'date'}, inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:
        users.append(entry[1])
        messages.append(" ".join(entry[2:]))
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)


df.head()

In [ ]:
df['month']=df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['year']=df['date'].dt.year
df.head()

In [ ]:
# list of userss
user_list = df['user'].unique().tolist()
user_list

In [ ]:
#no of messages by a user
df[df['user'] == 'user name'].shape[0]  
# give name of user in place of user name say maverick

In [ ]:
# total no. of words
words = []
for message in df['message']:
    words.extend(message.split())

len(words)

In [ ]:
## for media we will count media ommited

df[df['message'] == '<Media omitted>\n'].shape[0]

In [ ]:
# for links

from urlextract import URLExtract
extractor = URLExtract()

link = []

for message in df['message']:
    link.extend(extractor.find_urls(message))

link

In [ ]:
len(link) # no. of links

In [ ]:
import matplotlib.pyplot as plt

x = df['user'].value_counts().head()
name = x.index
count = x.values
plt.bar(name, count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# chat percentage
round(x/df.shape[0]*100,2).reset_index().rename(columns={'index':'name','user':'percent'})

In [ ]:
# for word cloud
from wordcloud import WordCloud

wc = WordCloud(width=600, height=500, min_font_size=10,
               background_color='white')
df_wc = wc.generate(df['message'].str.cat(sep=" "))
plt.figure(figsize=(18, 18))
plt.imshow(df_wc)

In [ ]:
words = []
for message in df['message']:
    words.extend(message.split())

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# we need to filter this
# here we need to remove :
#   media ommited
#   stop words
#   group messages

In [ ]:
temp = df[df['user'] != 'group_notification']
temp = temp[temp['message'] != '<Media omitted>\n']
temp

In [ ]:
# to prevent stopwords

f = open('stop_hinglish.txt', 'r')
stop_words = f.read()
words = []
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)

pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# for emojis
import emoji
emojis = []
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.UNICODE_EMOJI['en']])
pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))

In [ ]:
df['month_num'] = df['date'].dt.month
df.head()

In [ ]:
timeline = df.groupby(['year', 'month_num', 'month']).count()[
    'message'].reset_index()
timeline

In [ ]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))
time


In [ ]:
timeline['time'] = time
timeline

In [ ]:
plt.plot(timeline['time'], timeline['message'])
plt.xticks(rotation='vertical')

In [ ]:
df['daily_date']=df['date'].dt.date
daily_timeline = df.groupby('daily_date').count()['message'].reset_index()
daily_timeline

In [ ]:
plt.figure(figsize=(18, 10))
plt.plot(daily_timeline['daily_date'], daily_timeline['message'])

In [ ]:
df['day_name'] = df['date'].dt.day_name()
df.head()

In [ ]:
day_timeline = df.groupby('day_name').count()['message'].reset_index()
day_timeline

In [ ]:
plt.figure(figsize=(18, 10))
plt.bar(day_timeline['day_name'], day_timeline['message'])

In [ ]:
df.head(1)

In [ ]:
# creating the preiod list
period = []
for hour in df[['day_name', 'hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))

period

In [ ]:
df['period'] = period
df

In [ ]:
import seaborn as sns
plt.figure(figsize=(20, 6))
sns.heatmap(df.pivot_table(index='day_name', columns='period',
            values='message', aggfunc='count').fillna(0))
plt.yticks(rotation='horizontal')
plt.show()